# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
#
# Brief EDA
#

import pandas as pd

df = pd.read_csv('tic-tac-toe.csv')

display(df.head())

print("\n\nShape...")
display(df.shape)

print("\n\nSummary Statistics...")
display(df.describe())

print("\n\nClass Distribution...")
display(df["class"].value_counts())

print("\n\nMissing Values...")
display(df.isnull().sum())

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True




Shape...


(958, 10)



Summary Statistics...


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
count,958,958,958,958,958,958,958,958,958,958
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,True
freq,418,378,418,378,458,378,418,378,418,626




Class Distribution...


class
True     626
False    332
Name: count, dtype: int64



Missing Values...


TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

In [2]:
#
# EDA Summary:
#
# - 9 features, 1 target variable
# - Moderate class imbalance (65% win rate for PlayerX)
# - No missing values, datase is clean


In [3]:
# 
# Convert the categorical values to numeric in all columns.
# 

 
# For player moves, we can actually use ordinal encoding (1 and -1 will encode opposing players symetrically, while 0 will represent neutral/blank spaces):
#
# - x → -1 (PlayerX)
# - b → 0 (neutral / blank space)
# - o → 1 (PlayerY)

mapping = {"x": -1, "b": 0, "o": 1}
feat_cols = [c for c in df.columns if c != 'class']
df[feat_cols] = df[feat_cols].replace(mapping).astype(int)


# For the class column, we can use label encoding:
# 
# True → 1 (PlayerX wins)
# False → 0 (PlayerY wins)

df['class'] = df['class'].map({True: 1, False: 0}).astype(int)


# Check the updated DataFrame
display(df.head())


/var/folders/x1/00w2xr_j197gk698c1ljm3300000gn/T/ipykernel_4187/13180699.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[feat_cols] = df[feat_cols].replace(mapping).astype(int)


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,-1,-1,-1,-1,1,1,-1,1,1,1
1,-1,-1,-1,-1,1,1,1,-1,1,1
2,-1,-1,-1,-1,1,1,1,1,-1,1
3,-1,-1,-1,-1,1,1,1,0,0,1
4,-1,-1,-1,-1,1,1,0,1,0,1


In [11]:
# 
# Separate the inputs and output
# 

from sklearn.model_selection import train_test_split

X = df.drop('class', axis=1)
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
#
# Normalize the input data
#

# In this case, no need to normalize the data, since all features are on the same scale and small range (-1, 0, 1), we can feed them directly into the neural network.


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build the model
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(9,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))  # 2 outputs

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

# Save
model.save('tic-tac-toe.model.keras')


Epoch 1/20


/Users/luis/.pyenv/versions/3.10.13/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6126 - loss: 0.6573 - val_accuracy: 0.7403 - val_loss: 0.6035
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7116 - loss: 0.5966 - val_accuracy: 0.7468 - val_loss: 0.5776
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7264 - loss: 0.5743 - val_accuracy: 0.7532 - val_loss: 0.5610
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7645 - loss: 0.5422 - val_accuracy: 0.7597 - val_loss: 0.5493
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7403 - loss: 0.5412 - val_accuracy: 0.7727 - val_loss: 0.5374
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7517 - loss: 0.5209 - val_accuracy: 0.7727 - val_loss: 0.5244
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7262 - loss: 0.5357 - val_accuracy: 0.7792 - val_loss: 0.5147
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7689 - loss: 0.4805 - val_accuracy: 0.7792 - val_loss: 0.5043
Epo

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [21]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the saved model
model = load_model('tic-tac-toe.model.keras')

# Select a few random rows from X_test
random_indices = np.random.choice(X_test.index, size=5, replace=False)
X_sample = X_test.loc[random_indices]
y_true = y_test.loc[random_indices]

# Make predictions
pred_probs = model.predict(X_sample)

# Convert probabilities to class labels (0 or 1)
pred_classes = np.argmax(pred_probs, axis=1)

# Print results
for i, idx in enumerate(random_indices):
    print(f"Index: {idx}, True class: {y_true[idx]}, Predicted: {pred_classes[i]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Index: 107, True class: 1, Predicted: 1
Index: 78, True class: 1, Predicted: 1
Index: 316, True class: 1, Predicted: 0
Index: 891, True class: 0, Predicted: 1
Index: 830, True class: 0, Predicted: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Index: 107, True class: 1, Predicted: 1
Index: 78, True class: 1, Predicted: 1
Index: 316, True class: 1, Predicted: 0
Index: 891, True class: 0, Predicted: 1
Index: 830, True class: 0, Predicted: 0


In [22]:
# Display loss and accuracy

final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print(f"Train loss: {final_train_loss:.4f}, Train accuracy: {final_train_acc:.4f}")
print(f"Val loss: {final_val_loss:.4f}, Val accuracy: {final_val_acc:.4f}")


Train loss: 0.2787, Train accuracy: 0.9281
Val loss: 0.3324, Val accuracy: 0.8831


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

<br><br>


Let's test with different options for the number of layers...


In [ ]:
#
# Test different options for the number of layers
#

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
import numpy as np


def build_model(num_layers, input_shape=(9,)):
    model = Sequential()

    model.add(Input(shape=input_shape))
    model.add(Dense(32, activation='relu'))

    for _ in range(num_layers - 1):
        model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

layer_options = [1, 2, 3, 4, 5, 6, 8, 10, 12]
epochs = 20
batch_size = 32

results = []

for num_layers in layer_options:
    print(f"\nTraining model with {num_layers} hidden layer(s)...")
    model = build_model(num_layers)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_split=0.2, verbose=0)
    val_acc = history.history['val_accuracy'][-1]
    print(f"Validation accuracy: {val_acc:.4f}")
    results.append((num_layers, val_acc))



Training model with 1 hidden layer(s)...
Validation accuracy: 0.8182

Training model with 2 hidden layer(s)...
Validation accuracy: 0.8831

Training model with 3 hidden layer(s)...
Validation accuracy: 0.9740

Training model with 4 hidden layer(s)...
Validation accuracy: 0.9740

Training model with 5 hidden layer(s)...
Validation accuracy: 0.9675

Training model with 6 hidden layer(s)...
Validation accuracy: 0.9610

Training model with 8 hidden layer(s)...
Validation accuracy: 0.9740

Training model with 10 hidden layer(s)...
Validation accuracy: 0.9221

Training model with 12 hidden layer(s)...
Validation accuracy: 0.9026


<br>


Analysis:

- Four hidden layers seem optimal, as they yield the highest validation accuracy before performance plateaus or slightly declines, suggesting a good balance between model complexity and generalization.



<br><br>

Next, let's test different learning rates...



In [ ]:
#
# Test different options learning rate
#

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

learning_rates = [0.001, 0.005, 0.01, 0.02, 0.05]
results = []

for lr in learning_rates:
    print(f"\nTraining with learning rate: {lr}")
    model = Sequential([
        Input(shape=(9,)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(16, activation='relu'),
        Dense(16, activation='relu'),
        Dense(2, activation='softmax')
    ])

    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=0)
    
    val_acc = max(history.history['val_accuracy'])
    print(f"Best val_accuracy: {val_acc:.4f}")
    results.append((lr, val_acc))




Training with learning rate: 0.001
Best val_accuracy: 0.9675

Training with learning rate: 0.005
Best val_accuracy: 0.9935

Training with learning rate: 0.01
Best val_accuracy: 0.9870

Training with learning rate: 0.02
Best val_accuracy: 0.9935

Training with learning rate: 0.05
Best val_accuracy: 0.9805

Best learning rate: 0.005 with val_accuracy: 0.9935


<br>


Analysis:

- Results vary a lot each time we run these tests. This is explained due to the relatively small dataset size. 
- In general, all tested learning rates between 0.001 and 0.05 perform well.
- We'll choose a learning rate of 0.005, as it consistently achieved high validation accuracy across multiple runs.


<br><br>

Let's test different values for the number of epochs...


In [ ]:
#
# Test different options number of epochs
#

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import numpy as np

epoch_options = [5, 10, 20, 30, 40, 50, 60, 80, 100, 120, 150]

for epochs in epoch_options:
    model = Sequential()
    model.add(Input(shape=(9,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.005),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=epochs, batch_size=32,
              validation_split=0.2, verbose=0)

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Epochs: {epochs} - Test accuracy: {test_acc:.4f}")


Epochs: 5 - Test accuracy: 0.9479
Epochs: 10 - Test accuracy: 0.9792
Epochs: 20 - Test accuracy: 0.9844
Epochs: 30 - Test accuracy: 0.9792
Epochs: 40 - Test accuracy: 0.9896
Epochs: 50 - Test accuracy: 0.9688
Epochs: 60 - Test accuracy: 0.9792
Epochs: 80 - Test accuracy: 0.9792
Epochs: 100 - Test accuracy: 0.9896
Epochs: 120 - Test accuracy: 0.9792
Epochs: 150 - Test accuracy: 0.9792


Analysis:

- Once again, results vary a lot each time we run these tests. This is explained due to the relatively small dataset size. 
- In general, all values between 10 and 150 epochs perform well.
- We'll choose a number of epochs of 40, as it consistently achieved good results, provides reasonable training times and will generalize better than higher values.

<br><br>

Putting all together...

- 4 hidden layers
- 0.005 learning rate
- 40 epochs


In [60]:
#
# Recommended settings
#

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam


model = Sequential()
model.add(Input(shape=(9,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.005),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=40, batch_size=32,
            validation_split=0.2, verbose=0)


# Display loss and accuracy
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print(f"Train loss: {final_train_loss:.4f}, Train accuracy: {final_train_acc:.4f}")
print(f"Val loss: {final_val_loss:.4f}, Val accuracy: {final_val_acc:.4f}")

Train loss: 0.0001, Train accuracy: 1.0000
Val loss: 0.0630, Val accuracy: 0.9935


<br><br>


**Which approach(es) did you find helpful to improve your model performance?**

In [62]:

# The number of layers was the most impactful hyperparameter, with 4 hidden layers yielding the best results. 
